In [1]:
import numpy as np
from scipy import *
import os
import h5py
import keras as k
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from sklearn.utils import shuffle
from keras.utils import np_utils
import matplotlib.pyplot as plt
import scipy.optimize as opt
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean


%matplotlib inline

Using TensorFlow backend.


In [2]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'C:\\Users\\fires\\Desktop\\CSIRO\\thursday\\Logistic Regression\\data.h5', 'r')

images = np.asarray(data['images'])
labels = data['labels']

In [3]:
# Define function to crop off 'cut' number of pixels on both sides on both axixs
def crop_center(img,cut):
    y = np.shape(img)[1]
    x = np.shape(img)[2]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it ")
        
    start = cut
    end = x-cut
    return img[:, start:end,start:end]

In [4]:
# Reducing Image Size
images_cut = crop_center(images,66)
images_cut = np.asarray(images_cut)

print (images_cut.shape)

(356, 135, 135)


In [5]:
# Defining rescaling function
def rescale_imgs(images, factor):
    count =  0
    for i in range(0 ,images.shape[0]):
        img_scaled = expand_dims(rescale(images[i, :, :], 1.0 / float(factor)), axis = 0)
        if count == 0:
            imgs = img_scaled
        else:
            imgs = np.vstack((imgs, img_scaled))
        count += 1
                             
    return imgs

In [6]:
images_small = (rescale_imgs(images_cut, 5))

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fir

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fir

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fir

In [7]:
print (np.shape(images_small))

(356, 27, 27)


In [8]:
# Reshape data
img = np.reshape(images_small, (-1, 729))
lab = np.reshape(labels, (-1,1))

In [9]:
# balance classes
def bal_classes(images, labels):
    class1 = []
    class2 = []
    
    for i in range(labels.shape[0]):
        if labels[i] == False:
            class1.append(i)
    
    for i in range(labels.shape[0]):
        if labels[i] == True:
            class2.append(i)
    
    if len(class2) > len(class1):
        class2 = np.asarray(class2[:len(class1)])
    
    elif len(class2) < len(class1):
        class1 = np.asarray(class1[:len(class2)])
        
    images1 = images[class1, :] 
    images2 = images[class2, :]
    labels1 = labels[class1,]
    labels2 = labels[class2,]
    
    print (labels2.shape)
    
    images_st = np.vstack((images1, images2))
    labels_st =  np.concatenate((labels1, labels2), axis=0)              
        
    images_s, labels_s = shuffle(np.asarray(images_st), np.asarray(labels_st), random_state = 0)    
        
    return images_s, labels_s

In [10]:
images_p, labels_p = bal_classes(img, lab)

(123, 1)


In [11]:
labels = np.asarray(labels)
labels = np.reshape(labels, (-1,1))

In [12]:
#imgs = np.reshape(normalized_img, (-1, 27, 27))

#for n in range(0, imgs.shape[0]):
   #plt.imshow(imgs[n,:,:], cmap='gray', shape=(27, 27))
   #print (n)
   #print (normalized_labels[n])
   #plt.show()

In [13]:
# Split data into training and testing sets

ratio = 0.8

def format_data(images, labels):
    split = round(ratio*images.shape[0])
    
    train_x = images[ :split, :]
    test_x = images[split:, :]
    train_y = labels[ :split]
    test_y = labels[split:]
    
    return train_x, test_x, train_y, test_y


train_x, test_x, train_y, test_y = format_data(images_p, labels_p)

In [14]:
# Defining Sigmoid Function
def sigmoid(z):
    g = np.array([z]).flatten()
    g = 1.0/(1.0+np.exp(-g))
    return g

# Defining logistic loss function
def costJ(theta, x, y):
    m = x.shape[0]
    hypothesis = sigmoid(x.dot(theta).T)
    loss = J = -np.sum(y*np.log(hypothesis)+(1-y)*log(1-hypothesis))/m
    
    
    if np.isnan(loss) == True:
        loss = 0.0000000001
     
    return loss

def  grad(theta, x, y):
    #this returns a vector, which is the gradient of the cost function (partial derivatives of the cost function)
    m = x.shape[0]
    hypothesis = sigmoid(x.dot(theta).T)
    
    error = hypothesis-y
    
    gradient = []
    for i in range(x.shape[1]):
        gradient.append(np.sum(error*(x[:,i]))/m)
    return gradient
    
    assert grad.shape == (730,), grad.shape
    return grad    

# Defining prediction function 
def predict(theta, x):
    p_1 = sigmoid(x.dot(theta).T)
    return np.reshape((p_1 >= 0.5).astype(int), (-1,1))

# Defining accuracy function 
def accuracy(predictions, test_y):
    if predictions.shape != test_y.shape:
        print ("Error")
    else:
        test_y[test_y == True] = 1
        test_y[test_y == False] = 0

        correct_pred = 0

        for i in range(test_y.shape[0]):
            if test_y[i] == predictions[i]:
                correct_pred += 1

        acc = correct_pred/test_y.shape[0]
    return acc

In [15]:
def format_data(data):
    # m= number of training data, n= number of features
    m = data.shape[0]
    n = data.shape[1]

    #Add bias
    data = np.hstack((np.ones((m, 1)), data)) 
    
    return data


#initiate values for the inital theta (taking into acount bias)
initial_theta = np.zeros((train_x.shape[1] + 1,))

In [16]:
train_x.shape 

(197, 729)

In [17]:
# formating data
train_x = format_data(train_x)
test_x = format_data(test_x)

# Divide by max (reduce range to be between 0 and 1)
train_x /= np.max(train_x)
test_x /= np.max(test_x)


print (initial_theta.shape)

(730,)


In [18]:
costJ(initial_theta,train_x,train_y)

136.54999457030922

In [19]:
Result = opt.minimize(fun = costJ, x0 = initial_theta, args = (train_x, train_y), method = 'TNC', jac=grad)

In [20]:
g = predict(Result['x'], train_x)
accuracy(g, train_y)

0.5126903553299492